In [1]:
import sys
#!{sys.executable} -m pip install pafy
#!{sys.executable} -m pip install youtube-dl
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install facenet-pytorch

    100% |################################| 1.9MB 272kB/s eta 0:00:01    43% |#############                   | 819kB 4.7MB/s eta 0:00:01


In [1]:
import time
from os import getenv
import os
import io
from dotenv import load_dotenv
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets as widgets
import traitlets
import PIL
from PIL import ImageDraw
from IPython.display import display, Image
from jetcam.utils import bgr8_to_jpeg
import IPython
from jetcam.usb_camera import USBCamera
from facenet_pytorch import MTCNN
from IPython.display import Image
from my_heatmap_generator import HMap
import cv2

device = torch.device('cuda')
mtcnn = MTCNN(keep_all=True, device=device)
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
model.load_state_dict(torch.load('./model/my_model(1).pth'))
model.eval()
print("model loaded")
outframe = None
url = "https://youtu.be/7PYzSXHd6U4"
load_dotenv()

#defaults
active = "Live"
default_radius_size = getenv('DEFAULT_RADIUS')
default_duration = getenv('DEFAULT_DURATION')
radius = int(getenv('DEFAULT_RADIUS'))
duration = int(getenv('DEFAULT_DURATION'))

def on_value_change(change):
    print("Value change")
    radius = radius_slider.value
    duration = duration_slider.value

duration_slider = widgets.IntSlider(
    value=duration,
    min = 2,
    max = 30,
    step=1,
    description='Airborne Duration:',
    continuous_update=True,
    orientation='horizontal'
)

radius_slider = widgets.IntSlider(
    value=duration,
    min = 10,
    max = 200,
    step=1,
    description='Radius:',
    continuous_update=True,
    orientation='horizontal'
)

model loaded


In [2]:
HMAP = []
camera = USBCamera(width=224, height=224, capture_fps=30, capture_device=0)
camera.running = True
camera.value
print("camera created")

def preprocess(image):
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

image_widget = widgets.Image(format='jpeg', width=224, height=224)

# Draw polygon with radial gradient from point to the polygon border
# ranging from color 1 to color 2 on given image
def radial_gradient(frame, box, radius):
    #overlay_image = PIL.Image.new("RGB", frame.size, color="red")  # This could be a bitmap fill too, but let's just make it orange
    #img = PIL.Image.new("L", frame.size, color=0) 
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((box[0]-radius, box[1]-radius, box[2]+radius, box[3]+radius), fill=(255,0,0,100))
    
    c1 = (255, 0, 0)
    c2 = (0, 255, 0)
    p = (((box[0]+box[2])/2), (box[1]+box[3])/2))

    # Draw initial polygon, alpha channel only, on an empty canvas of image size
    ii = Image.new('RGBA', frame.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(ii)
    draw.ellipse((box[0]-radius, box[1]-radius, box[2]+radius, box[3]+radius), fill=(0, 0, 0, 255), outline=None)

    # Use polygon vertex with highest distance to given point as end of gradient
    max_dist = radius

    # Calculate color values (gradient) for the whole canvas
    x, y = np.meshgrid(np.arange(frame.size[0]), np.arange(frame.size[1]))
    c = np.linalg.norm(np.stack((x, y), axis=2) - p, axis=2) / max_dist
    c = np.tile(np.expand_dims(c, axis=2), [1, 1, 3])
    c = (c1 * (1 - c) + c2 * c).astype(np.uint8)
    c = Image.fromarray(c)

    # Paste gradient on temporary image
    ii.paste(c, mask=ii)
    mask_image = ii.filter(ImageFilter.GaussianBlur(radius=10))
    frame = PIL.Image.composite(ii, frame, mask_image)
    return frame


def execute(change):
    global hmap, radius
    frame = change['new']
    frame = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    width, height = frame.size[:2]
    boxes, _ = mtcnn.detect(frame)
    if boxes is not None:
        for box in boxes:
            rectangle = (box[0]-20, box[1]-20, box[2]+20, box[3]+20)
            face_only = frame.crop(rectangle)
            face_only = face_only.resize((224, 224))
            processed = preprocess(face_only)
            output = model(processed)
            output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
            category_index = output.argmax()
            draw = ImageDraw.Draw(frame, 'RGBA')
            x = (box[0] + box[2])/2
            y = (box[1] + box[3])/2
            radius = radius_slider.value
            frame = radial_gradient(frame, box, radius)
    buf = io.BytesIO()
    frame.save(buf, format='JPEG')
    image_widget.value = buf.getvalue()

camera.observe(execute, names='value')
display_handle = display(image_widget, widgets.HBox([duration_slider, radius_slider]))

camera created


Image(value=b'', format='jpeg', height='224', width='224')

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg/jetcam/camera.py", line 34, in _capture_frames
    self.value = self._read()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 588, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 577, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1210, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1215, in notify_change
    return self._notify_observers(change)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", l